In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import torch

In [11]:
# -*- coding: utf-8 -*-
"""tinygptforcollab

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/19DiOjHtL-UEa69WlOcMsmRnkrNc5NmaS
"""

# -*- coding: utf-8 -*-
"""tinygpt.ipynb
Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/109yvdricQdl9D43KPCeh829Y89TGUcvC


"""

import torch
device='cuda' if torch.cuda.is_available() else 'cpu'
eval_iters=200
n_embd=384
lr=3e-4
max_iters=5000
batch_size=64
block_size=256
num_heads=6
eval_inter=500
dropout=0.2
nlayerb=6

chars=['\n', ' ', '!', '"', '#', '$', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '\xad', '´', 'é', 'ñ', '\u200a', '\u200b', '–', '—', '‘', '’', '“', '”', '…', '🎓']
vocab_size=len(chars)


encode = lambda e: [stoi[c] for c in e]

itos={i:s for i, s in enumerate(chars)}
stoi={s:i for i, s in enumerate(chars)}
decode = lambda d: "".join([itos[c] for c in d])








import torch
from torch import nn as nn
import torch.nn.functional as F

class Attention(nn.Module):
   def __init__(self):
    super().__init__()
    self.kqv = nn.Linear(n_embd, 3*n_embd)
    self.ddp=nn.Dropout(dropout)
    self.proj=nn.Linear(n_embd, n_embd)


   def forward(self, x):
      B, T, C = x.shape
      k, q, v = self.kqv(x).split(n_embd, dim=2)
      k = k.view(B, T, num_heads, C//num_heads).transpose(1, 2)
      q = q.view(B, T, num_heads, C//num_heads).transpose(1, 2)
      v = v.view(B, T, num_heads, C//num_heads).transpose(1, 2)
      y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=dropout if self.training else 0, is_causal=True)
      y = y.transpose(1, 2).contiguous().view(B, T, C)
      out=self.ddp(self.proj(y))
      return out




class feedfoward(nn.Module):
  def __init__(self, n_embd):
    super().__init__()
    self.net=nn.Sequential(nn.Linear(n_embd, 4*n_embd), nn.ReLU(), nn.Linear(4*n_embd, n_embd), nn.Dropout(dropout))
  def forward(self, x):
    ffd=self.net(x)
    return ffd

class Block(nn.Module):
  def __init__(self, n_embd, num_heads):
    super().__init__()
    head_size=n_embd//num_heads
    self.heads=Attention()
    self.ffd=feedfoward(n_embd)
    self.lln=nn.LayerNorm(n_embd)
  def forward(self, x):
    x=x+self.heads(self.lln(x))
    x=x+self.ffd(self.lln(x))
    return x







torch.manual_seed(1337)
class BigramModel(nn.Module):
    def __init__ (self):
        super().__init__()
        self.tokenembeddingtable=nn.Embedding(vocab_size, n_embd)
        self.positionembeddingtable=nn.Embedding(block_size, n_embd)
        self.heads=Attention()
        self.ffd=feedfoward(n_embd)
        self.block=nn.Sequential(*[Block(n_embd, num_heads=num_heads)for _ in range(nlayerb)])
        self.lnf=nn.LayerNorm(n_embd)
        self.lmhead=nn.Linear(n_embd, vocab_size)
    def forward(self, idx, targets=None):
        idx = idx.to(device)
        B, T= idx.shape
        tok_emb=self.tokenembeddingtable(idx)
        pos_emb=self.positionembeddingtable(torch.arange(T, device=device))
        x=tok_emb+pos_emb
        x=self.block(x)
        x=self.lnf(x)

        logits=self.lmhead(x)


        B, T, C =logits.shape

        if targets is not None:
          logits=logits.view(B*T, C)
          targets=targets.to(device)

          targets=targets.view(B*T)
          loss=F.cross_entropy(logits, targets)
        else:
          loss=None
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
          idx_cong=idx[:, -block_size:]
          logits, loss=self(idx_cong)
          logits=logits[:, -1, :]
          prob=F.softmax(logits, dim=-1)
          idx_next=torch.multinomial(prob, num_samples=1)
          idx=torch.cat((idx, idx_next), dim=1)


        return idx













bm1=BigramModel()
bm1=bm1.to(device)
bm1.eval()
bm1.load_state_dict(torch.load('/content/drive/MyDrive/tinygpttinystories.pt'))







<All keys matched successfully>

In [12]:
prompt='Once' #prompt from source code
#prompt=input('enter prompt to start generation:') #optional
f=torch.tensor(encode(prompt), device=device)
f=f.view(1, len(prompt))
n_of_tokens=10000 #increase n_0f_tokens for more examples
print(decode(bm1.generate(idx = f,  max_new_tokens=n_of_tokens)[0].tolist()))

Once upon a time, there lived a big tree with lots of help frosting so new light. Then, they set out of maes and shouting of the tree tasks right to the frog.
One day, the little boy wanted a band with a smile. He sang and took a snack. And the little boy wanted to never would have out of the exam wooden to find such a new friend wobbed the magic away!
He felt scared friend. Eventually, he saw some kart on a dance with a smile. He closed the magic and continued in the air, go wood in the head. He tried to fix the grandma, but it wood wouldn't fill the leaf.
The next day fixed the small dance, the magic grandma piece had to wander a nice lady of this wisdom important fixed with dolls. Mom scared her view beautifuld and danced, and came out on a few wastest filled in a little world a loud crazy nap.
After this first, Lisa grew bigger filled the brothes were. She surprised of walk to Ben and her friends. They rolled back and her parents and looked up to their hammeding.
<|endoftext|>
Sue 